# Otsuka - Accumulations

In [4]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [5]:
# Read pdf into DataFrame
df = tabula.read_pdf("2019-Otsuka-OPCH-Switzerland-Final-Report-EFPIA-12062019-German_2235.pdf", pages='all', lattice=True)

## Format Table

In [14]:
df_export = df.copy()

#Shift
column0 = df_export[df_export.columns[0]]
df_export[column0.notna()] = df_export[column0.notna()].shift(1, axis='columns')

#Drop Columns
df_export = df_export.drop(df_export.columns[9:],axis=1)
df_export = df_export.drop(df_export.columns[[0]],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

# Select
amount = df_export[df_export.type.str.contains('Gesamtbetrag zurechenbarer geldwerter ', na=False)]
recipients = df_export[df_export.type.str.contains('Gesamtzahl der Empfänger', na=False)]
rnd = df_export[df_export.type.str.contains('Geldwerte Leistungen im Zusammenhang', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Numberize
df_export = remove_comma(df_export)
df_export = df_export.replace("Optional", np.NaN)
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

#fix rnd
df_export.loc[df_export.type == 'rnd', "donations_grants"] = np.NaN

#Remove Count Total
df_export.loc[df_export['type'].str.contains('count'), 'total'] = np.nan

#Export
export_acumulations(df_export, 'otsuka')

saved


In [15]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')